##Test TikTokScalping

https://www.tiktok.com/@eduscalping/video/7358462952246103302

In [217]:
import pandas as pd
import numpy as np
from datetime import timedelta

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 

from pandas_ta.momentum import rsi as ta_rsi
from pandas_ta.momentum import stoch as ta_stoch
from pandas_ta.momentum import macd as ta_macd

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


#Desicion de operacion en Largo
- RSI inferior a 35, sobre de 45, aumentando constante
- Estocastico Inferior 20 y sobre 35, aumentando constante
- MACD cruzo hacia arriba

#Desicion de operacion en Corto
- RSI Supero 65, por debajo de 55, disminuyendo constante
- Estocastico Supero 80 y por debajo de 75, disminuyendo constante
- MACD cruzo hacia abajo

In [218]:
klines_file = './backtest/klines/0m05/Lateral_TRXUSDT_0m05_2023-06-20_2023-10-09.DataFrame'
klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
klines_file  = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)

#df = fn.ohlc_mirror_v(df)

#ini = pd.to_datetime('2023-08-03')
#fin = pd.to_datetime('2023-08-08')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
df.drop(columns=['volume'],inplace=True)


In [219]:
#RSI
df['rsi'] = ta_rsi(df['close'],length=14)

#Estocastico
stoch = ta_stoch(df['high'],df['low'],df['close'])
df['stoch'] = stoch['STOCHk_14_3_3']
#df['stoch'] = stoch['STOCHd_14_3_3']

#MACD
macd = ta_macd(df['close'])
df['macd'] = macd['MACD_12_26_9']
df['macd_h'] = macd['MACDh_12_26_9']
df['macd_s'] = macd['MACDs_12_26_9']

df['rsi_buy'] = None
df['rsi_sell'] = None
df['rsi_top'] = False
df['rsi_bot'] = False

df['stoch_buy'] = None
df['stoch_sell'] = None
df['stoch_top'] = False
df['stoch_bot'] = False

df['macd_buy'] = None
df['macd_sell'] = None

df['buy'] = None
df['sell'] = None

df['ops'] = 0

ok = False
ops = 0
for i, row in df.iterrows():
    if ok:
        if row['rsi']> 50:
            if (pre_row['rsi']<65 and row['rsi']>65) or (pre_row['rsi_top']):
                df.at[i,'rsi_top'] = True
                if row['rsi']<65:
                    df.at[i,'rsi_sell'] = row['rsi']
        else:
            if (pre_row['rsi']>35 and row['rsi']<35) or (pre_row['rsi_bot']):
                df.at[i,'rsi_bot'] = True
                if row['rsi']>35:
                    df.at[i,'rsi_buy'] = row['rsi']

        if row['stoch']> 50:
            if (pre_row['stoch']<80 and row['stoch']>80) or (pre_row['stoch_top']):
                df.at[i,'stoch_top'] = True
                if row['stoch']<80:
                    df.at[i,'stoch_sell'] = row['stoch']
        else:
            if (pre_row['stoch']>20 and row['stoch']<20) or (pre_row['stoch_bot']):
                df.at[i,'stoch_bot'] = True
                if row['stoch']>20:
                    df.at[i,'stoch_buy'] = row['stoch']
        
        if row['macd'] > row['macd_s'] and row['macd'] > 0:
            df.at[i,'macd_sell'] = row['macd']
        if row['macd'] < row['macd_s'] and row['macd'] < 0:
            df.at[i,'macd_buy'] = row['macd']

        row = df.loc[i]

        if not pre_row['buy'] and row['rsi_buy'] and row['stoch_buy'] and row['macd_buy']:
            df.at[i,'buy'] = row['close']
            ops += 1

        if not pre_row['sell'] and row['rsi_sell'] and row['stoch_sell'] and row['macd_sell']:
            df.at[i,'sell'] = row['close']
            ops -= 1

    df.at[i,'ops'] = ops  
    
    pre_row = df.loc[i]
    ok = True


In [220]:
indicators_out = [
         {'col': 'rsi_sell','name': 'RSI','color': 'red','row': 2,},
         {'col': 'rsi_buy','name': 'RSI','color': 'green','row': 2,},         
         {'col': 'stoch_sell','name': 'Stoch D','color': 'red','row': 3,},
         {'col': 'stoch_buy','name': 'Stoch D','color': 'green','row': 3,},
         {'col': 'macd_sell','name': 'MACD','color': 'red','row': 4,},
         {'col': 'macd_buy','name': 'MACD','color': 'green','row': 4,},
     ]
indicators_out = [
         {'col': 'ops','name': 'OPS','color': 'yellow','row': 3,},
         {'col': 'macd','name': 'MACD','color': 'blue','row': 4,},
        ]
events = [
         {'df':df,
          'col':'buy',
          'name': 'BUY',
          'color': 'green',
          'symbol': 'circle-dot' #https://plotly.com/python/reference/scatter/#scatter-marker-symbol
         },
         {'df':df,
          'col':'sell',
          'name': 'SELL',
          'color': 'red',
          'symbol': 'circle-dot' #https://plotly.com/python/reference/scatter/#scatter-marker-symbol
         },
     ]
fig = fn.ohlc_chart(df,events=events,indicators_out=indicators_out) #
fig.show()